In [1]:
%pylab inline
import bokeh
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, Range1d, LabelSet, Label
output_notebook()

from glob import glob
import gensim
from nltk.corpus import stopwords

from sklearn.decomposition import PCA

import torch
from torch import nn
from torch.nn import functional as F
from torch import optim

from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader

from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Accuracy
from time import time
from pathlib import Path

Populating the interactive namespace from numpy and matplotlib


Loading BokehJS ...

In [2]:
corpus, target = [], []

stopw = stopwords.words("english")
for file in glob("dataset/*labelled.txt"):
    texts = open(file).read()
    i = 0
    for line in texts.splitlines():
        
        try:
            target.append(int(line.split("\t")[-1]))
            
            sc = gensim.utils.simple_preprocess(line) # sentence candidate
            sc = [i for i in sc if i not in stopw]
            corpus.append(sc)
        except:
            print(file, i)
        i += 1

assert len(corpus) == len(target), f"corpus: {len(corpus)} | target: {len(target)}"

dataset/imdb_labelled.txt 178
dataset/imdb_labelled.txt 968


In [3]:
max_sentence = 0
for i in corpus:
    len_sentence = len(i)
    if len_sentence > max_sentence:
        max_sentence = len_sentence
print("max_sentence: ", max_sentence)

max_sentence:  44


In [4]:
m = max_sentence
n = 10

In [5]:
skip_gram = gensim.models.Word2Vec(corpus, size=n, sg=1, iter=2000, min_count=1)

In [6]:
id2word = skip_gram.wv.index2word
word2id = {k:v for v, k in enumerate(id2word)}

In [7]:
pca = PCA(n_components=2)
XY = pca.fit_transform(skip_gram[skip_gram.wv.vocab])

/home/lincoln/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [8]:
source = ColumnDataSource(data=dict(x=XY[:, 0], y=XY[:, 1], labels=list(skip_gram.wv.vocab.keys())))
p = figure(plot_width=600, plot_height=600)
p.scatter(x='x', y='y', size=8, source=source)
labels = LabelSet(x='x', y='y', text='labels', level='glyph',
              x_offset=5, y_offset=5, source=source, render_mode='canvas')

p.add_layout(labels)
show(p)

In [9]:
del XY, source, p, labels, pca

In [10]:
def make_matrix(phrase, model_vec, word2id, stop_words):
    if type(phrase) is str:
        phrase = gensim.utils.simple_preprocess(phrase)
        phrase = [i for i in phrase if i not in stop_words]
        phrase = [i for i in phrase if i in model_vec.wv.index2word]
    
    phrase_ids = [word2id[i] for i in phrase]
    
            

    return phrase_ids

In [11]:
make_matrix(corpus[442], skip_gram, word2id, stopw)

[806, 2420, 92, 95, 326, 324]

In [12]:
class Data(Dataset):
    def __init__(self, data, target, model, word2id, stop_words):
        
        self.data =  [make_matrix(i, model, word2id, stop_words) for i in data]
        self.target =  torch.LongTensor([int(i) for i in target])
        self._len = len(target)
        
    def __getitem__(self, x):
        return torch.LongTensor(self.data[x]), self.target[x]
    
    def __len__(self):
        return self._len

In [13]:
data = Data(corpus, target, skip_gram, word2id, stopw)

In [14]:
data[4][1]

tensor(1)

In [15]:
train_data, test_data, train_target, test_target = train_test_split(corpus, target, test_size=0.25)

data_train = Data(train_data, train_target, skip_gram, word2id, stopw)
data_test = Data(test_data, test_target, skip_gram, word2id, stopw)

In [16]:
print(f"\
lengh train: {data_train._len}\n\
lengh test: {data_test._len}")

lengh train: 2250
lengh test: 750


---
according to: https://medium.com/@martinpella/how-to-use-pre-trained-word-embeddings-in-pytorch-71ca59249f76

---

In [32]:
class RNN(nn.Module):
    def __init__(self, vocab_size, model, m, n, n_layers, hidden, out, mode):
        super(RNN, self).__init__()
        
        self.n_layers = n_layers
        self.m = m
        self.n = n
        self.hidden_size = hidden
        self.mode = mode
        
        self.embed = nn.Embedding(vocab_size, n)
        self.embed.load_state_dict({
            "weight": torch.FloatTensor(model.wv.vectors)            
        })
        
        if mode is "gru":
            self.recurrence = nn.GRU(n, hidden, n_layers, dropout=0.2)
        elif mode is "lstm":
            self.recurrence = nn.LSTM(n, hidden, n_layers, dropout=0.2)
        else:
            raise "escolha entre gru e lstm apenas"
        
        self.linear0 = nn.Linear(m*hidden, int(hidden*2))
        self.linear1 = nn.Linear(int(hidden*2), hidden)
        self.linear2 = nn.Linear(hidden, out)
        
    def forward(self, inpt):
        
        hidden = self._init_hidden(inpt.size(1))
        
        x, hidden = self.recurrence(
            self.embed(inpt),
            hidden
        )
        
        space = torch.zeros(self.m*self.hidden_size)
        x = x.flatten()
        space[:x.shape[0]] = x
        x = space
        
        x = self.linear0(x)
        x = self.linear1(x)
        x = self.linear2(x)
        
        return torch.sigmoid(x).unsqueeze(0)
    
    def _init_hidden(self, batch_size):
        if self.mode == "lstm":
            return (torch.zeros(self.n_layers, batch_size, self.hidden_size),
                    torch.zeros(self.n_layers, batch_size, self.hidden_size))
        else:
            return torch.zeros(self.n_layers, batch_size, self.hidden_size)
        

In [33]:
rnn = RNN(len(id2word), skip_gram, m, n, 2, 16, 2, mode="lstm")
t = next(iter(DataLoader(data_train)))
print(rnn(t[0]))
del t

tensor([[0.5034, 0.5328]], grad_fn=<UnsqueezeBackward0>)


In [34]:
train_loader = DataLoader(data_train, batch_size=1, shuffle=True)
test_loader = DataLoader(data_test,  batch_size=1)

In [35]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(rnn.parameters(), lr=1e-4)

trainer = create_supervised_trainer(
    rnn, optimizer, criterion
)

evaluator = create_supervised_evaluator(
    rnn,
    metrics={
        "accuracy": Accuracy()
    },
    non_blocking=True
)

In [36]:
@trainer.on(Events.STARTED)
def start_prepare(trainer):
    trainer._loss = []
    trainer._tmp_loss = []
    trainer._last_loss = None
    trainer._accuracy_train = []
    trainer._accuracy_test = []
    trainer._init_time = time()
    trainer._csv_path = "results.csv"
    if not Path(trainer._csv_path):
        with open(trainer._csv_path, "x") as f:
            f.write("epoch, loss, accuracy_train, accuracy_test, model\n")
            f.close()

@trainer.on(Events.ITERATION_COMPLETED)
def mean_loss(trainer):
    trainer._tmp_loss.append(trainer.state.output)
            
@trainer.on(Events.EPOCH_COMPLETED)
def registry_progress(trainer):
    evaluator.run(train_loader)
    
    trainer._last_loss = np.mean(trainer._tmp_loss)
    trainer._tmp_loss = []
    
    loss = trainer._last_loss
    trainer._loss.append(
        loss
    )
    accuracy_train = evaluator.state.metrics["accuracy"]
    trainer._accuracy_train.append(
        accuracy_train
    )
    
    evaluator.run(test_loader)
    accuracy_test = evaluator.state.metrics["accuracy"]
    trainer._accuracy_test.append(
        accuracy_test
    )

    with open(trainer._csv_path, "a") as f:
        f.write(f"{trainer.state.epoch},{loss},{accuracy_train},{accuracy_test},rnn_lstm\n")
        f.close()

@trainer.on(Events.EPOCH_COMPLETED)
def progress_log(trainer):
    if trainer.state.epoch % 5 == 0:
        evaluator.run(train_loader)
        metrics = evaluator.state.metrics
        print(f"epoch: {trainer.state.epoch:<2} | " + \
              f"accuracy: {metrics['accuracy']:.2f} | " + \
              f"loss: {trainer._loss[-1]:.2f}")
              
@trainer.on(Events.COMPLETED)
def summary(trainer):
    print(f"\n\
    ---\n\
    epochs: {trainer.state.epoch}\n\
    total time: {time() - trainer._init_time:.0f} seconds\n\n\
    \t\t loss \t train \t test \n\
    init\t {trainer._loss[0]:.3f} \t {trainer._accuracy_train[0]:.3f} \t {trainer._accuracy_test[0]:.3f}\n\
    end\t {trainer._loss[-1]:.3f} \t {trainer._accuracy_train[-1]:.3f} \t {trainer._accuracy_test[-1]:.3f}\n\
    ---")
    torch.save(rnn, "saved_models/rnn_lstm.pth")


In [37]:
state = trainer.run(train_loader, max_epochs=20)

epoch: 5  | accuracy: 0.74 | loss: 0.57
epoch: 10 | accuracy: 0.76 | loss: 0.55
epoch: 15 | accuracy: 0.79 | loss: 0.53
epoch: 20 | accuracy: 0.80 | loss: 0.51

    ---
    epochs: 20
    total time: 1469 seconds

    		 loss 	 train 	 test 
    init	 0.692 	 0.684 	 0.656
    end	 0.514 	 0.801 	 0.751
    ---


/home/lincoln/miniconda3/lib/python3.7/site-packages/torch/serialization.py:250: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


''

In [40]:
figsize(15, 7)

plot_loss = plot(trainer._loss, label="loss")
plot_train = plot(trainer._accuracy_train, label="accuracy train")
plot_test = plot(trainer._accuracy_test, label="accuracy test")
plt.legend()
plt.title("rnn lstm")
;